# Датасет от компании разработчика мобильных игр. 

In [1]:
import pandas as pd
import seaborn as sns
from operator import attrgetter

import matplotlib.pyplot as plt
from datetime import datetime

import numpy as np

In [2]:
file_1 = '/mnt/HC_Volume_18315164/home-jupyter/jupyter-m-podmarev/shared/problem1-reg_data.csv'

In [3]:
file_2 = '/mnt/HC_Volume_18315164/home-jupyter/jupyter-m-podmarev/shared/problem1-auth_data.csv'

In [4]:
# данные о времени регистрации
reg = pd.read_csv(file_1, sep = ';')

In [5]:
# данные о времени захода пользователей в игру
auth = pd.read_csv(file_2, sep = ';')

<h3>Данные о времени регистрации</h3>

In [6]:
# данные о времени регистрации
# визуально как выглядит таблица
reg.head(5)

,reg_ts,uid
0,911382223,1
1,932683089,2
2,947802447,3
3,959523541,4
4,969103313,5


In [7]:
rows, columns = reg.shape
print(f'В датасете: ')
print(f'    {rows:,} строк')
print(f'    {columns} колонки')

В датасете: 
    1,000,000 строк
    2 колонки


In [8]:
# ура нету пропусков
reg.isna().sum()

reg_ts    0
uid       0
dtype: int64

<h3>Данные о времени захода пользователей в игру</h3>

In [9]:
# данные о времени захода пользователей в игру
auth.head(5)

,auth_ts,uid
0,911382223,1
1,932683089,2
2,932921206,2
3,933393015,2
4,933875379,2


In [10]:
rows, columns = auth.shape
print(f'В датасете: ')
print(f'    {rows:,} строк')
print(f'    {columns} колонки')

В датасете: 
    9,601,013 строк
    2 колонки


In [11]:
# ура нету пропусков 2
auth.isna().sum()

auth_ts    0
uid        0
dtype: int64

In [12]:
auth.groupby('uid', as_index=False).agg({'auth_ts':'count'}).describe()

,uid,auth_ts
count,1.000000e+06,1000000.000000
mean,5.552354e+05,9.601013
std,3.206011e+05,47.069539
min,1.000000e+00,1.000000
25%,2.776438e+05,1.000000
50%,5.550455e+05,1.000000
75%,8.329712e+05,1.000000
max,1.110622e+06,1929.000000


# Exploratory Data Analysis

<strong>Uid</strong> общая колонка для обоих датасэтов, это игроки-пользователи. Поэтому хотелось бы посмотреть насколько они совпадают

In [13]:
unique_users_1 = reg.uid.unique()
unique_users_2 = auth.uid.unique()
print(f'Количество уникальных игроков в 1-ом датасете = {len(unique_users_1):,}') 
print(f'Количество уникальных игроков в 2-ом датасете = {len(unique_users_2):,}')
print(f'Они совпадают? {set(unique_users_1) == set(unique_users_2)}')


Количество уникальных игроков в 1-ом датасете = 1,000,000
Количество уникальных игроков в 2-ом датасете = 1,000,000
Они совпадают? True


# Задача №1: посчитать retention игроков (по дням от даты регистрации игрока)

<strong>Retention Rate</strong> — коэффициент удержания клиентов. Он показывает, сколько пользователей возвращается в мобильное приложение. 
1. Note: Разные специалисты считают коэффициент по-разному в зависимости от информации, которую хотят получить.\
    1.1 В нашем случае по дням от даты регистрации игрока

Икея такая : Retention вычисляем как 
   - Периодичность: дни
   - Сравнение: Дата Аутентификации к Дате Регистрации   

In [16]:
# Retention Rate function 

def retention(date_start='1998-11-18',  # дата откуда считаем Retention - дата регистрации
              date_to='2020-09-23',     # дата докуда считаем Retention - дата регистрации
              num_of_days_you_need = 7, # сколько дней отобразить в output таблице, по умолчанию 7 дней
              reg=reg,                  # сами таблицы
              auth=auth):                
    
    # to convert to days both Dataframes
    if reg['reg_ts'].dtypes == 'int64':
        reg['reg_ts'] = pd.to_datetime(reg['reg_ts'], unit='s').dt.to_period('D')
    elif auth['auth_ts'].dtypes == 'int64':
        auth['auth_ts'] = pd.to_datetime(auth['auth_ts'], unit='s').dt.to_period('D')
    
    # filter from and to the dates you need
    reg = reg[(reg['reg_ts'] >= date_start) & (reg['reg_ts'] <= date_to)]
    
    # merge tables for a common df
    ret_curve = auth.merge(reg, how='inner', on='uid')
    
    # build cohorts
    cohorts = ret_curve.groupby(['reg_ts', 'auth_ts'], as_index=False).agg(user_count=('uid', 'nunique'))
    # days of retention
    cohorts['Day'] = (cohorts.auth_ts - cohorts.reg_ts).apply(attrgetter('n'))
    # filter for how many days to display
    cohorts_filter = cohorts.query('Day < @num_of_days_you_need + 1')
    # pivot table of cohorts
    cohorts_pivot = cohorts_filter.pivot_table(index='reg_ts', columns='Day', values='user_count')\
                    .rename_axis(['Day of Registration:\nYY-MM-DD'])\
                    .rename(columns=lambda x: f'Day {x}')
    
    # ограничения на размер когорты и считаем retention
    cohort_size = cohorts_pivot.iloc[:, 0]
    retention_rate = cohorts_pivot.divide(cohort_size, axis=0)
    retention_rate = retention_rate.iloc[:, 0:(30)]
    # Формат отображения значений
    retention_rate_formatted = retention_rate.applymap(lambda x: '{:.2%}'.format(x))
       
    return retention_rate_formatted


In [17]:
# 1-й аргумент: Вы можете выбрать начальную дату регистрации (точка отсчета)
# 2-й аргумент: Вы можете выбрать конечную дату регистрации (точка конца)
# Даты нужно подставить в формате YY-MM-DD

# 3-й аргумент: сколько дней отобразить в графике от даты регистрации (необзяательно, по умолчанию 7)
retention('2020-01-01', '2020-01-07')

Day,Day 0,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7
Day of Registration: YY-MM-DD,,,,,,,,
2020-01-01,100.00%,2.64%,4.52%,3.39%,5.66%,5.84%,7.54%,5.00%
2020-01-02,100.00%,2.35%,4.52%,5.17%,4.80%,5.83%,7.71%,5.93%
2020-01-03,100.00%,2.35%,3.38%,4.41%,4.51%,6.85%,7.51%,5.45%
2020-01-04,100.00%,1.50%,4.50%,5.06%,5.34%,5.72%,7.03%,5.44%
2020-01-05,100.00%,1.96%,4.49%,4.58%,4.40%,4.77%,7.86%,5.24%
2020-01-06,100.00%,2.90%,4.49%,4.95%,3.83%,4.67%,6.64%,5.89%
2020-01-07,100.00%,1.87%,4.85%,4.38%,6.44%,5.04%,8.49%,5.13%


<h1> Задача №2: A/B тест</h1>
Имеются результаты A/B теста, в котором двум группам пользователей предлагались различные наборы акционных предложений.

Известно, что ARPU в тестовой группе выше на 5%, чем в контрольной. \
При этом в контрольной группе 1928 игроков из 202103 оказались платящими, а в тестовой – 1805 из 202667.

# Какой набор предложений можно считать лучшим? Какие метрики стоит проанализировать для принятия правильного решения и как?

In [ ]:
from urllib.parse import urlencode
import requests

In [ ]:
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://disk.yandex.ru/d/SOkIsD5A8xlI7Q' 
final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']
download_response = requests.get(download_url)

In [ ]:
ab = pd.read_csv(download_url, sep = ';')

In [ ]:
ab.head()

In [ ]:
rows, columns = ab.shape
print(f'В датасете: ')
print(f'    {rows:,} строк')
print(f'    {columns} колонки')
print(f'     ')

print(f'В таблице {ab.user_id.nunique()} уникальных юзеров')
print(f'В таблице {ab["user_id"].duplicated().sum()} дупликатов')



In [ ]:
ab.revenue.info()

In [ ]:
ab.groupby('testgroup').size()

<h3>Инфо о группах</h3>

- <h1>Группа а - контрольная:</h1> \
         - 1,928 игроков из 202,103 оказались платящими
            
            
- <h1>Группа b - тестовая:</h1> \
        - 1,805 из 202,667 оказались платящими\
        - ARPU в тестовой группе выше на 5%

<strong>ARPU</strong> - Average revenue per user \
Average Revenue Per User = весь доход / количество пользователей за период



In [ ]:
# доход общий
total_revenue = ab.revenue.sum()
print(f'Общий доход {total_revenue:,}')

In [ ]:
# доход Группа A
a_revenue = ab[ab['testgroup'] == 'a'].revenue.sum()
print(f'Доход с группы А: {a_revenue:,}')

In [ ]:
# доход Группа B
b_revenue = ab[ab['testgroup'] == 'b'].revenue.sum()
print(f'Доход с группы B: {b_revenue:,}')

In [ ]:
control_a = ab[ab['testgroup'] == 'a']
test_b = ab[ab['testgroup'] == 'b']

In [ ]:
ARPU_a = len(ab[ab['testgroup'] == 'a']) / (ab[ab['testgroup'] == 'a'].revenue.sum())
ARPU_b = len(ab[ab['testgroup'] == 'b']) / (ab[ab['testgroup'] == 'b'].revenue.sum())

print(f'ARPU группы А: {round(ARPU_a, 3)}')
print(f'ARPU группы B: {round(ARPU_b, 3)}')

In [ ]:
#  ARPU в тестовой группе ДЕЙСТВИТЕЛЬНО выше на 5%
round(ARPU_b / ARPU_a * 100, 2)

In [ ]:
# сколько плятщих 1,928
control_a.query('revenue != 0').shape

In [ ]:
# сколько плятщих 1,805
test_b.query('revenue != 0').shape

<h3>Какой набор предложений можно считать лучшим? Какие метрики стоит проанализировать для принятия правильного решения и как?
</h3>


<h3>1. ARPU (Средний доход на пользователя):</h3>

Это метрика, которая уже была указана, и она выше на 5% в тестовой группе.

In [ ]:
#  ARPU в тестовой группе ДЕЙСТВИТЕЛЬНО выше на 5%
round(ARPU_b / ARPU_a * 100, 2)

<h3>2. Конверсия (Conversion Rate):</h3>

Процент пользователей, которые совершают покупку в каждой группе.

In [ ]:
control_cr = int(control.query('revenue != 0').agg({'user_id':'count'})) / int(control.agg({'user_id':'count'}))

In [ ]:
test_cr = int(test.query('revenue != 0').agg({'user_id':'count'})) / int(test.agg({'user_id':'count'}))

In [ ]:
print(f"Conversion Rate for Control group А = {control_cr:%}")
print(f"Conversion Rate for Test group В = {test_cr:%}")
# Процент пользователей, которые совершают покупку выше в Группе А (контрольной)

<h3>Средний доход на платящего пользователя (ARPPU - Average Revenue Per Paying User):</h3>

Эта метрика показывает, сколько в среднем приносит один платящий пользователь.


In [ ]:
arppu_control = int(control.query('revenue != 0').agg({'revenue':'sum'})) / int(control.query('revenue != 0').agg({'user_id':'count'}))

In [ ]:
arppu_test = int(test.query('revenue != 0').agg({'revenue':'sum'})) / int(test.query('revenue != 0').agg({'user_id':'count'}))

In [ ]:
print(f"ARPPU for Control group = {round(arppu_control, 2)}")
print(f"ARPPU for Test group = {round(arppu_test, 2)}")
# ARPPU выше в Тестовой группе (В)

<h3>P-value:</h3>

Статистическая значимость различий между группами.

In [ ]:
test_pay = test_b.query('revenue != 0')
control_pay = control_a.query('revenue != 0')

In [ ]:
# посмотрим по числам, группа A
control_pay.revenue.describe()

In [ ]:
# посмотрим по числам, группа В
test_pay.revenue.describe()

Размер выборки примерно одинаков 1928 к 1805\
Средние близки 2663 к 3003\
Стандартное отклонение СИЛЬНО различается 9049 к 572\
Квартили сильно разнятся\
В группе А ОГРОМНЫЙ выброс в максимуме.

In [ ]:
# визуально
sns.set_style("whitegrid")
plt.figure(figsize=(10, 6))

In [ ]:
# Create the boxplot with customizations
ax = sns.boxplot(
    data=ab.query('revenue != 0'),
    x='testgroup',
    y='revenue',
    palette='Set2'  # Choose a color palette
)

# Add titles and labels
ax.set_title('Revenue Distribution by Groups BoxPlots', fontsize=16)
ax.set_xlabel('Test Group', fontsize=14)
ax.set_ylabel('Revenue', fontsize=14)

# Customize the ticks
ax.tick_params(axis='x', labelsize=12)
ax.tick_params(axis='y', labelsize=12)

# Optionally, add a horizontal grid to improve readability
ax.yaxis.grid(True, linestyle='--', which='both', color='grey', alpha=0.7)

# Remove the top and right spines for a cleaner look
sns.despine(trim=True)

# Show the plot
plt.show()

In [ ]:
# группа А
control_pay.revenue.describe()

In [ ]:
from scipy.stats import mannwhitneyu

In [ ]:
test_pay_revenue = test_pay.revenue
control_pay_revenue = control_pay.revenue

У нас есть огромные выбросы - поэтому лучше будет сравнить выборки тестом Манна-Уитни.

In [ ]:
stat, p_value = mannwhitneyu(test_pay_revenue, control_pay_revenue, alternative='two-sided')

print(f'Statistics: {stat}, p-value: {p_value}')

P-value < 0.05, поэтому можно отвергать нулевую гипотезу о сходстве двух выборок. Однако мы это и визуально видим, что выборки сильно различаются - в особенности мешают выбросы в группе А.\
Посмотрим сколько выбросов вообще

In [ ]:
# С доходом выше 35тыс (выбросы) у нас 123 юзера!
control_a.query('revenue > 35000')

In [ ]:
print(f' Получается доля юзеров заплативших более 35тыс составила = {(123 / 1928):%}')

Случайность это или все дело в особенностях контрольной группы А и его набора акционных предложений - сказать сложно.\
Может быть у приложения есть доля юзеров готовых платить значительные суммы и они также случайно могли оказаться в группе В,
а может быть это следствие акционных предложений тестировавшихся в группе А.

<h2>Что мы можем сделать в таком случае? Что бы решить какой вариант лучше</h2>

<h3>1. Мы можем УБРАТЬ выбросы</h3>

In [ ]:
ab_no_outlier = ab.query('revenue != 0 and revenue < 35000')

In [ ]:
ax = sns.boxplot(
    data=ab_no_outlier,
    x='testgroup',
    y='revenue',
    palette='Set2'  # Choose a color palette
)

# Add titles and labels
ax.set_title('Revenue Distribution by Groups Boxplots without Outliers', fontsize=16)
ax.set_xlabel('Test Group', fontsize=14)
ax.set_ylabel('Revenue', fontsize=14)

# Customize the ticks
ax.tick_params(axis='x', labelsize=12)
ax.tick_params(axis='y', labelsize=12)

# Optionally, add a horizontal grid to improve readability
ax.yaxis.grid(True, linestyle='--', which='both', color='grey', alpha=0.7)

# Remove the top and right spines for a cleaner look
sns.despine(trim=True)

# Show the plot
plt.show()

И визуально видим что распределение доходов лучше в группе В.\
<strong>ЕСЛИ мы думаем что выбросы случайны и не зависят от акционных предложений группы.</strong>

<h3>2. Мы можем сделать бутстрап, посмотрев на медианы</h3>
Давайте все же условимся что выбросы - случайны.

In [ ]:
revenue_control_a = control_a.revenue
revenue_test_b    = test_b.revenue

In [ ]:
def bootstrap(data, num_iterations, statistic):
    bootstrapped_stats = []
    for _ in range(num_iterations):
        sample = np.random.choice(data, size=len(data), replace=True)
        bootstrapped_stats.append(statistic(sample))
    return bootstrapped_stats

In [ ]:
# Данные для бутстрапа
control_data_no_outliers = control_a.query('revenue !=0 and revenue <= 35000').revenue.values
control_data_outliers = control_a.query('revenue !=0').revenue.values

test_data = test_b.query('revenue !=0').revenue.values

#control_data = control_a.revenue.values
#test_data    = test_b.revenue.values

In [ ]:
# Количество итераций
num_iterations = 1000

In [ ]:
# Бутстрап для среднего значения
control_bootstrap_means_no_outliers = bootstrap(control_data_no_outliers, num_iterations, np.mean)
control_bootstrap_means_outliers = bootstrap(control_data_outliers, num_iterations, np.mean)
test_bootstrap_means = bootstrap(test_data, num_iterations, np.mean)


control_bootstrap_means = bootstrap(control_data, num_iterations, np.mean)

In [ ]:
# Доверительные интервалы
control_ci_no_outliers = np.percentile(control_bootstrap_means_no_outliers, [2.5, 97.5])
control_ci_outliers = np.percentile(control_bootstrap_means_outliers, [2.5, 97.5])
test_ci = np.percentile(test_bootstrap_means, [2.5, 97.5])

In [ ]:
print("Control group mean Confidence Interval with outliers:",    control_ci_outliers)
print("Control group mean Confidence Interval without outliers:", control_ci_no_outliers)
print(" ")
print("Test group mean Confidence Interval:", test_ci)

<h2>Интерпретация результатов:</h2>
Доверительные интервалы средних значений для контрольной и тестовой групп не перекрываются, это свидетельствует о статистически значимом различии.


<h3>Если выбросы не являются случайными и являются следствием особенностей группы, то их игнорирование может привести к упущению важной информации. В этом случае необходимо тщательно рассмотреть, как включение выбросов повлияет на результаты анализа.
</h3>


# Итог: 
1. Если считать что выбросы - случайны. То однозначно следует выбрать группу В. Так как она показала повышение распределения доходов от пользователей.
2. Если выбросы - не случайны. То следует провести другой тест который должен включать в тестовую группу пользователей которые платят очень много.